In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

os.chdir(f'/content/drive/MyDrive/Polimi/NLP')
os.getcwd()

'/content/drive/MyDrive/Polimi/NLP'

# Data upload and import libreries

In [4]:
!pip install -q "transformers" "peft" "accelerate" "bitsandbytes" "trl" "safetensors" "tiktoken"
!pip install -q -U langchain
!pip install -q langchain-community langchain-core
!pip install -q --no-deps peft
!pip install -q lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, pipeline
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
from peft import LoraConfig
from peft import prepare_model_for_kbit_training, get_peft_model
from datasets import Dataset
from torch.utils.data import DataLoader
import lightning as L
from torch.optim import AdamW
import torch.nn.functional as F
from huggingface_hub import login

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
import string
import re
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
import string
import re
from collections import Counter
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Load of the data

In [7]:
train_set = pd.read_parquet("tuning_data.parquet")
test_set = pd.read_parquet("test_data.parquet")
small_test_set = test_set[:15]

# Load of the model

Here we load the model and his tokenizer.

In [8]:
model_id = "NousResearch/Llama-2-7b-chat-hf"

The model is too big, so we will make use of a 4-bit version of the model.

In [9]:
# BitsAndBytesConfig int-4 config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

Load the model

In [10]:
fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Load the weights

In [11]:
state_dict = torch.load("model2_Llama.pth", map_location="cuda")
fine_tuned_model.load_state_dict(state_dict, strict=False)
fine_tuned_model.to("cuda")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

Load the tokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [13]:
# Prompt format for LLaMA-2 chat models
llama_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Testing

Let's test the fine tuned model and see if there are any improvements.

We first load our model with the weights.

In [14]:
generation_args_fine_tuned = {
    "max_new_tokens": 3000,
    "temperature": 0.1,
    "do_sample": True,
}

Define the function that generates and returns the answer.The genereted answer may include not only the direct answer but also additional context,
explanations, or formatting.

In [15]:
def chatbot_fine_tuned(prompt, tokenizer, model, generation_args):
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate output
    outputs = model.generate(**inputs, **generation_args_fine_tuned)

    # Decode generated tokens
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Define a function that extracts only the answer to the question from the generated response.

In [16]:
def extract_answer_only(generated_text):
    # Cerca il delimitatore "### Response:" e restituisce solo ciò che viene dopo
    if "### Response:" in generated_text:
        return generated_text.split("### Response:")[1].strip()
    elif "[/INST]" in generated_text:
        return generated_text.split("[/INST]")[-1].strip()
    else:
        return generated_text.strip()

Test on few samples.

In [24]:
n = 1

for i, row in test_set.head(n).reset_index(drop=True).iterrows():
    question = row['question']
    context = row['context']
    target_answer = row['answer']

    prompt = llama_prompt.format(question, context, "")

    # Generate the answer
    result = chatbot_fine_tuned(prompt, tokenizer, fine_tuned_model, generation_args_fine_tuned)
    clean_answer = extract_answer_only(result)

    # Output
    print(f"Example {i+1}:")
    print("Question:", question)
    print("\nTarget answer:", target_answer)
    print("\nGenerated answer:", clean_answer)
    print("-" * 40)

Example 1:
Question: Who is the music director of the Quebec Symphony Orchestra?

Target answer: The music director of the Quebec Symphony Orchestra is Fabien Gabel.

Generated answer: Fabien Gabel is the music director of the Quebec Symphony Orchestra.
----------------------------------------
